In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline
import numpy as np

from gala.mpl_style import hesperia, laguna, turbo_r

from thriftshop.data import load_apogee_sample
from thriftshop.config import plot_config as pc
from thriftshop.config import fig_path

coord.galactocentric_frame_defaults.set('v4.0');

In [ ]:
allstar = at.Table.read('/Users/apricewhelan/data/APOGEE_beta/allStarLite-r13-l33-58932beta.fits')

Apply the same selections we apply in `select_apogee_sample.py` before the M/H Alpha/M polygon:

In [ ]:
aspcap_bitmask = np.sum(2**np.array([
    7,  # STAR_WARN
    23  # STAR_BAD
]))
qual_mask = (
    (allstar['LOGG'] < 3.5) &
    (allstar['LOGG'] > 1) &
    (allstar['TEFF'] < 6500) &
    (allstar['TEFF'] > 3500) &
    (allstar['SNR'] > 40) &
    (allstar['FE_H'] > -3) &
    (allstar['MG_FE'] > -1) &
    ((allstar['ASPCAPFLAG'] & aspcap_bitmask) == 0)
)
stars = allstar[qual_mask]
_, idx = np.unique(stars['APOGEE_ID'], return_index=True)
stars = stars[idx]

# Remove stars targeted in known clusters
mask_bits = {
    'APOGEE_TARGET1': np.array([9, 18, 24, 26]),
    'APOGEE_TARGET2': np.array([10, 18]),
    'APOGEE2_TARGET1': np.array([9, 18, 20, 21, 22, 23, 24, 26]),
    'APOGEE2_TARGET2': np.array([10]),
    'APOGEE2_TARGET3': np.array([5, 14, 15])
}
target_mask = np.ones(len(stars), dtype=bool)
for name, bits in mask_bits.items():
    target_mask &= (stars[name] & np.sum(2**bits)) == 0
    
gaia_mask = (stars['GAIA_PARALLAX'] > 0.4) & ((stars['GAIA_PARALLAX'] / stars['GAIA_PARALLAX_ERROR']) > 5)
    
full = stars[target_mask & gaia_mask]
len(full)

In [ ]:
t, c = load_apogee_sample('../../data/apogee-parent-sample.fits')
c = c[(t['GAIA_PARALLAX'] / t['GAIA_PARALLAX_ERROR']) > 5]
t = t[(t['GAIA_PARALLAX'] / t['GAIA_PARALLAX_ERROR']) > 5]

In [ ]:
mh_alpham_nodes = np.array([[0.6, -0.05],
                            [0.6, 0.04],
                            [0.15, 0.04],
                            [-0.5, 0.13],
                            [-0.9, 0.13],
                            [-1., 0.07],
                            [-0.2, -0.1],
                            [0.2, -0.1],
                            [0.6, -0.05]])

In [ ]:
vstep = 2.
zstep = 50 / 1e3
vzz_bins = (np.arange(-pc['vlim'], pc['vlim']+1e-3, vstep),
            np.arange(-pc['zlim'], pc['zlim']+1e-3, zstep))

In [ ]:
norm = mpl.colors.LogNorm(vmin=1, vmax=5e2)
cmap = 'hesperia_r'

fig = plt.figure(figsize=(12, 10),
                 constrained_layout=True)

gs = GridSpec(nrows=2, ncols=2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

bins = (np.arange(-2., 0.75, 0.01),
        np.arange(-0.18, 0.4, 0.005))
H, xe, ye = np.histogram2d(full['M_H'], full['ALPHA_M'], bins=bins)
H_sub, xe, ye = np.histogram2d(t['M_H'], t['ALPHA_M'], bins=bins)

ax1.pcolormesh(xe, ye, H.T, norm=norm, cmap='Greys', rasterized=True)
cs = ax1.pcolormesh(xe, ye, H_sub.T, norm=norm, cmap=cmap, rasterized=True)

ax1.plot(mh_alpham_nodes[:, 0], mh_alpham_nodes[:, 1],
         marker='', color='#777777', ls='--', lw=1.5)
ax1.set_xlim(-1.5, 0.7)
ax1.set_ylim(-0.12, 0.32)
ax1.set_xlabel('metallicity, [M/H]')
ax1.set_ylabel(r'alpha abundance, [$\alpha$/M]')

# ---

x = c.galactocentric.x.to_value(u.kpc)
y = c.galactocentric.y.to_value(u.kpc)
z = c.galactocentric.z.to_value(u.kpc)
vz = c.galactocentric.v_z.to_value(u.km / u.s)

bins = (np.arange(-8.122 - 2.6, -8.122 + 2.6 + 1e-3, zstep),
        np.arange(-2.6, 2.6 + 1e-3, zstep))
H, xe, ye = np.histogram2d(x, y, bins=bins)
ax2.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

ax2.set_xlabel('Galactocentric $x$ [kpc]')
ax2.set_ylabel('Galactocentric $y$ [kpc]')

H, xe, ye = np.histogram2d(vz, z, bins=vzz_bins)
ax3.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

ax3.set_xlabel(f'Galactocentric $v_z$ [{u.km/u.s:latex_inline}]')
ax3.set_ylabel('Galactocentric $z$ [kpc]')

cb = fig.colorbar(cs, ax=ax1, aspect=30)
cb.set_label('$N$ stars')

ax1.set_title('APOGEE–Gaia Parent Sample', fontsize=24, pad=12)

# fig.tight_layout()
fig.set_facecolor('w')

fig.savefig(fig_path / 'mh-am-xy.pdf', dpi=300)

In [ ]:
fig = plt.figure(figsize=(14., 4.5),
                 constrained_layout=True)

gs = GridSpec(nrows=1, ncols=18, figure=fig)
ax1 = fig.add_subplot(gs[:10])
ax2 = fig.add_subplot(gs[10:])

bins = (np.arange(-2.5, 0.75, 0.01),
        np.arange(-0.18, 0.4, 0.006))
H, xe, ye = np.histogram2d(full['M_H'], full['ALPHA_M'], bins=bins)

norm = mpl.colors.LogNorm(vmin=1, vmax=1e3)
cmap = 'hesperia_r'
ax1.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap)

ax1.plot(mh_alpham_nodes[:, 0], mh_alpham_nodes[:, 1],
         marker='', color='#777777', ls='--', lw=1.5)
ax1.set_xlim(-2.5, 0.7)
ax1.set_xlabel('metallicity, [M/H]')
ax1.set_ylabel(r'alpha abundance, [$\alpha$/M]')

# ---

x = c.galactocentric.x.to_value(u.kpc)
y = c.galactocentric.y.to_value(u.kpc)
bins = (np.arange(-8.122 - 2.6, -8.122 + 2.6 + 1e-3, 0.075),
        np.arange(-2.6, 2.6 + 1e-3, 0.075))
H, xe, ye = np.histogram2d(x, y, bins=bins)
cs = ax2.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap)
# ax2.set_aspect('equal')

ax2.set_xlabel('Galactocentric $x$ [kpc]')
ax2.set_ylabel('Galactocentric $y$ [kpc]')

cb = fig.colorbar(cs, ax=[ax1, ax2], aspect=40)
cb.set_label('$N$ stars')

fig.suptitle('APOGEE Parent Sample', fontsize=24)